In [2]:
# set up
#################
import numpy as np
import openpyxl
import pandas as pd


# iris data
def iris_df() -> pd.DataFrame:
    from sklearn.datasets import load_iris

    _ = load_iris()
    data = pd.DataFrame(
        _.data,
        columns=_.feature_names,
    )
    data["target"] = _.target
    data["target_category"] = data["target"].map(
        {
            0: "setosa",
            1: "versicolor",
            2: "virginica",
        }
    )
    data.columns = [x.replace(" ", "_") for x in data.columns]
    data.columns = [x.replace("(", "") for x in data.columns]
    data.columns = [x.replace(")", "") for x in data.columns]
    return data


iris = iris_df()

# pandas example table
data = pd.DataFrame(
    np.random.randn(10, 4),
    columns=list("ABCD"),
)

# make an excel with the table
data.to_excel(f"table.xlsx", index=True)

In [ ]:
# conditional formatting on existing excel
################################################

wb = openpyxl.load_workbook(f"{here}/table.xlsx")

ws = wb.active

red_text = openpyxl.styles.Font(color="00FF0000")

for row in ws.iter_rows(min_row=2, min_col=2, max_col=5, max_row=11):
    for cell in row:
        if cell.value < 0:
            cell.font = red_text

wb.save(f"{here}/table.xlsx")

In [ ]:
# conditional formatting on a dataframe then save to excel
########################################################

# work thru the dataframe, create cells with the color formatting

wb = openpyxl.Workbook()
ws = wb.active

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        cell = ws.cell(row=i + 2, column=j + 2)
        if data.iloc[i, j] < 0:
            cell.font = red_text
        cell.value = data.iloc[i, j]

wb.save(f"{here}/table2.xlsx")

In [ ]:
# use color scales to show the range of values, red to green
############################################################

wb = openpyxl.Workbook()
ws = wb.active

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        cell = ws.cell(row=i + 2, column=j + 2)
        cell.value = data.iloc[i, j]

ws.conditional_formatting.add(
    f"B2:E{data.shape[0]+1}",
    openpyxl.formatting.rule.ColorScaleRule(
        start_type="min",
        start_color="00FF0000",
        end_type="max",
        end_color="0000FF00",
    ),
)

wb.save(f"{here}/table3.xlsx")


# functionalise, need all numeric data
def save_to_excel_with_color_scale(data: pd.DataFrame, filename):
    wb = openpyxl.Workbook()
    ws = wb.active

    # fill in the data
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            cell = ws.cell(row=i + 2, column=j + 2)
            cell.value = data.iloc[i, j]

    # add the color scale
    ws.conditional_formatting.add(
        f"B2:E{data.shape[0]+1}",
        openpyxl.formatting.rule.ColorScaleRule(
            start_type="min",
            start_color="00FF0000",
            end_type="max",
            end_color="0000FF00",
        ),
    )

    wb.save(filename)


data = pd.DataFrame(
    np.random.randn(10, 4),
    columns=list("ABCD"),
)

save_to_excel_with_color_scale(data=data, filename=f"{here}/table4.xlsx")

In [ ]:
# add a chart
#############

import openpyxl as xl

data = iris
wb = openpyxl.Workbook()
ws = wb.active
sheetname = "iris"
ws.title = sheetname

# write col name row
for i in range(data.shape[1]):
    ws.cell(row=1, column=i + 2).value = data.columns[i]

# append each row of the dataframe to the sheet, including the index
for i in range(data.shape[0]):
    ws.cell(row=i + 2, column=1).value = data.index[i]
    for j in range(data.shape[1]):
        ws.cell(row=i + 2, column=j + 2).value = data.iloc[i, j]

# todo: make above into a function: dataframe_to_excel(data, filename, sheetname, index=True)


# calculate the table area in A1 notation
def get_table_area(data: pd.DataFrame, include_index=True) -> str:
    if include_index:
        min_col = 1
    else:
        min_col = 2
    max_col = data.shape[1] + 1
    max_row = data.shape[0] + 1
    min_row = 1
    top_left_letter = openpyxl.utils.get_column_letter(min_col)
    down_right_letter = openpyxl.utils.get_column_letter(max_col)
    return f"{top_left_letter}{min_row}:{down_right_letter}{max_row}"


get_table_area(data=data, include_index=True)


def close_open_to_close_close(
    start: int,
    end: int,
) -> tuple:
    """
    close_open_to_close_close(0, 0)
    close_open_to_close_close(0, 1)
    close_open_to_close_close(0, 2)
    close_open_to_close_close(1, 3)
    """
    if start >= end:
        return None
    else:
        return start, end - 1


# column start and end pass 1 number and data, output the range
def get_column_range(
    data: pd.DataFrame, start: int, end: int, include_index=True
) -> str:
    # convert to excel index
    start, end = start + 1, end
    if include_index:
        start += 1
        end += 1
    top_left_letter = openpyxl.utils.get_column_letter(start)
    down_right_letter = openpyxl.utils.get_column_letter(end)
    min_row = 1
    max_row = data.shape[0] + 1
    return f"{top_left_letter}{min_row}:{down_right_letter}{max_row}"


get_column_range(data=data, start=1, end=4)

# add the chart
chart = openpyxl.chart.LineChart()
chart.title = "Iris sepal length vs other measures"
chart.style = 13  # use a preset style: https://openpyxl.readthedocs.io/en/stable/charts/pie.html#pie-chart-styles
chart.x_axis.title = "sepal length (cm)"
chart.y_axis.title = "other measures"
chart.legend.position = "r"  # legend on the right

# values are the lines to plot
_ = get_column_range(data=data, start=1, end=4)
values = xl.chart.Reference(ws, range_string=f"{ws.title}!{_}")
_ = get_column_range(data=data, start=0, end=1)
x_values = xl.chart.Reference(ws, range_string=f"{ws.title}!{_}")

chart.add_data(values, titles_from_data=True)
chart.set_categories(x_values)

# add the chart to the sheet
ws.add_chart(chart, "H2")

wb.save(f"{here}/table5.xlsx")


# add xlsx with chart to existing csv
######################################

# save a csv
data = iris
data.to_csv(f"{here}/iris.csv")


# func
def add_line_chart(
    ws: openpyxl.worksheet.worksheet.Worksheet,
    values_range: str,
    x_values_range: str,
    title: str,
    x_axis_title: str,
    y_axis_title: str,
    legend_position: str,
    chart_style: int,
    chart_location: str,
):
    chart = openpyxl.chart.LineChart()
    chart.title = title
    chart.style = chart_style
    chart.x_axis.title = x_axis_title
    chart.y_axis.title = y_axis_title
    chart.legend.position = legend_position

    # values are the lines to plot
    values = openpyxl.chart.Reference(ws, range_string=values_range)
    x_values = openpyxl.chart.Reference(ws, range_string=x_values_range)

    chart.add_data(values, titles_from_data=True)
    chart.set_categories(x_values)

    # add the chart to the sheet
    ws.add_chart(chart, chart_location)


import csv


def copy_csv_into_ws(
    ws: openpyxl.worksheet.worksheet.Worksheet,
    csv_path: str,
):
    with open(csv_path, "r") as f:
        for row in csv.reader(f):
            cleaned = []
            for i in row:
                try:
                    cleaned.append(float(i))
                except:
                    cleaned.append(i)
            ws.append(cleaned)


# make new workbook
wb = openpyxl.Workbook()

# make new sheets
ws = wb.create_sheet("iris")

# add the csv to the sheet
copy_csv_into_ws(
    ws=ws,
    csv_path=f"{here}/iris.csv",
)

# make another sheets
ws2 = wb.create_sheet("iris2")
copy_csv_into_ws(
    ws=ws2,
    csv_path=f"{here}/iris.csv",
)

# another charts sheet
ws3 = wb.create_sheet("charts")

# add the chart
add_line_chart(
    ws=ws3,
    values_range=f"{ws.title}!C1:E151",
    x_values_range=f"{ws.title}!B1:B151",
    title="Iris sepal length vs other measures",
    x_axis_title="sepal length (cm)",
    y_axis_title="other measures",
    legend_position="r",
    chart_style=13,
    chart_location="A1",
)

add_line_chart(
    ws=ws3,
    values_range=f"{ws2.title}!C1:E151",
    x_values_range=f"{ws2.title}!B1:B151",
    title="Iris sepal length vs other measures",
    x_axis_title="sepal length (cm)",
    y_axis_title="other measures",
    legend_position="r",
    chart_style=13,
    chart_location="A10",
)

wb.save(f"{here}/charts.xlsx")